# Rotation Image From Boxes

In [4]:
%cd /content/drive/MyDrive/CRAFT_Text_Detection

/content/drive/MyDrive/CRAFT_Text_Detection


In [5]:
import cv2
from google.colab.patches import cv2_imshow
import torch
from PIL import Image
import numpy as np
from CRAFT import CRAFTModel, draw_boxes, draw_polygons, boxes_area, polygons_area

In [7]:
model = CRAFTModel('cpu', use_refiner=True, fp16=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/content/drive/MyDrive/CRAFT_Text_Detection/CRAFT/craft.py:90: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True

In [6]:
# Read image
img = cv2.imread('/content/drive/MyDrive/CRAFT_Text_Detection/test/mcocr_val_145114ixmyt.jpg')

In [ ]:
#Get boxes from model
boxes = model.get_boxes(img)
print(boxes)

In [ ]:
# Rotation image
def rotate_img(img, boxes):
  boxes_ratio = []
  direct_rotate = []
  for box in boxes:
    x1, y1, x2, y2 = box[0][0], box[0][1], box[1][0], box[1][1]
    box_ratio = abs(y1 - y2) / abs(x1 - x2)
    direct_rotate.append(y1 - y2)
  mean_ratio = np.mean(box_ratio)
  direction = np.mean(direct_rotate)
  #Radian Angle
  radian_angle = np.arctan(mean_ratio)
  if direction >= 0:
    degree_angle = -np.degree(radian_angle)
  else:
    degree_angle = np.degree(radian_angle)
  # Rotate image
  w,h  = img.shape[:2]
  (cX, cY) = (w // 2, h // 2)
  M = cv2.getRotationMatrix2D((cX, cY), degree_angle, 1.0)
  rotated_img = cv2.warpAffine(img, M, (w, h))

  return rotated_img